In [ ]:
import scqubits as scq
import numpy as np
import plotly.graph_objects as go
import matplotlib.pyplot as plt

from zcu_tools.analysis.fluxdep import load_result
from zcu_tools.datasaver import load_data

In [ ]:
qub_name = "S001"

In [ ]:
loadpath = f"../result/{qub_name}/params.json"
_, params, cflx, period, allows = load_result(loadpath)

if "r_f" in allows:
    r_f = allows["r_f"]
    print(f"r_f = {r_f}")


def phi2mA(ratio):
    return (period * (ratio - 0.5) + cflx) * 1e3


def mA2phi(mA):
    return (mA / 1e3 - cflx) / period + 0.5

In [ ]:
def calculate_dispersive(flxs, r_f, g):
    fluxonium = scq.Fluxonium(
        *params, flux=0.5, cutoff=50, truncated_dim=15, id_str="qubit"
    )
    resonator = scq.Oscillator(r_f, truncated_dim=10, id_str="resonator")
    hilbertspace = scq.HilbertSpace([fluxonium, resonator])
    hilbertspace.add_interaction(
        g=g,
        op1=fluxonium.n_operator,
        op2=resonator.creation_operator,
        add_hc=True,
        id_str="q-r coupling",
    )

    def update_hilbertspace(flux):
        fluxonium.flux = flux

    sweep = scq.ParameterSweep(
        hilbertspace,
        {"flxs": flxs},
        update_hilbertspace=update_hilbertspace,
        evals_count=20,
        subsys_update_info={"flxs": [fluxonium]},
    )

    return sweep["chi"]["subsys1":0, "subsys2":1].toarray()

# Plot Prediction

In [ ]:
# r_f = 5.000

flxs = np.linspace(0, 0.51, 501)
mAs = phi2mA(flxs)

In [ ]:
predict_chi = calculate_dispersive(flxs, r_f, 0.1)

In [ ]:
fig = go.Figure()

valid_num = np.sum(np.all(~np.isnan(predict_chi), axis=0))

for i in range(1, valid_num):
    fig.add_scatter(
        x=flxs,
        y=predict_chi[:, i] * 1e3,
        mode="lines",
        name=f"$\\chi_0{i}$",
    )

# plot a dash hline to indicate the 0 point, also add a xaxis2 to show mA
fig.add_scatter(
    x=mAs, y=np.zeros_like(mAs), xaxis="x2", line=dict(color="black", dash="dash")
)
mAs_ticks = mAs[:: len(mAs) // 10]
fig.update_layout(
    xaxis2=dict(
        tickvals=mAs_ticks,
        ticktext=[f"{mA:.2f}" for mA in mAs_ticks],
        overlaying="x",
        side="top",
        title_text="mA",
    )
)

fig.update_layout(
    xaxis_title=r"$Φ_{ext}/Φ_0$",
    yaxis_title=r"$\chi (MHz)$",
    legend_title="Dispersive",
    margin=dict(l=0, r=0, t=0, b=0),
)
fig.update_yaxes(range=[-2, 2])

fig.show()

In [ ]:
fig.write_html(f"../result/{qub_name}/web/dispersive.html", include_plotlyjs="cdn")
fig.write_image(
    f"../result/{qub_name}/image/dispersive.png", format="png", width=800, height=400
)

# Plot with Onetone

In [ ]:
onetone_path = "../Database/S001/2025/03/Data_0311/S001_OneTone_flux_003.hdf5"

signals, sp_fpts, sp_mAs = load_data(
    onetone_path, server_ip="005-writeboard", port=4999
)
sp_fpts = sp_fpts * 1e-9  # to GHz
sp_mAs = sp_mAs * 1e3  # to mA

In [ ]:
sp_flxs = mA2phi(sp_mAs)

predict_chi = calculate_dispersive(sp_flxs, r_f, 0.02)

In [ ]:
fig, ax = plt.subplots()


ax.imshow(
    np.abs(signals),
    extent=[sp_mAs[0], sp_mAs[-1], sp_fpts[0], sp_fpts[-1]],
    aspect="auto",
    interpolation="none",
    origin="lower",
)
ax.plot(sp_mAs, r_f + predict_chi[:, 1], color="blue", lw=2)

ax.set_xlabel("mA")
ax.set_ylabel("Frequency (GHz)")

plt.show()